In [1]:
import numpy as np
import os
import pandas as pd
import sox
import wave
import logging
import pydub 
from pydub.playback import play
import time
import csv
import glob
import shutil
import json

In [2]:
from IPython.display import clear_output

In [3]:
clips_path = "../mozilla_common_voice/clips/" # english corpus
df = pd.read_csv("../mozilla_common_voice/train.tsv", sep="\t")
print("number of mp3s in training set: ", df.shape[0])

number of mp3s in training set:  232975


### Generating a micro dataset of 50 clips

In [15]:
selected_clips=[]

In [15]:
def listen_to_sample(df, ix, clips_path):
    mp3_path = clips_path + df.iloc[ix].path
    clip = pydub.AudioSegment.from_mp3(mp3_path)
    play(clip)
    return clip

In [ ]:
sample = np.random.randint(df.shape[0])
listen_to_sample(df=df, ix=sample, clips_path=clips_path)
selected_clips.append(sample)
print(len(selected_clips))

In [ ]:
raise ValueError("warning: overwrites")
df.iloc[selected_clips].to_csv(
    "selected_clips.tsv", sep="\t", quoting=csv.QUOTE_MINIMAL, index=False
)

In [17]:
selected = pd.read_csv("selected_clips.tsv", sep="\t")

In [26]:
for _, r in selected.iterrows():
    mp3_path = clips_path + r.path
    shutil.copy2(mp3_path, "./micro_dataset")

Now, run `word_separator/transcribe_word_time_offsets.py` on `./micro_dataset`

### Split word on boundaries

In [7]:
selected = pd.read_csv("selected_clips.tsv", sep="\t")

In [4]:
# TODO(MMAZ) lower() and remove punctuation, etc
# pasted from https://stackoverflow.com/a/32558749
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [5]:
def listen_to_split_on_boundaries(timings, clip):
    for ix, timing in enumerate(timings):
        word = timing['word']
        start_ms = timing['start_time'] * 1000
        end_ms = timing['end_time'] * 1000
        print(f"word {ix}: {word}")
        play(clip[start_ms:end_ms])
        time.sleep(1)

In [27]:
for ix, row in selected.iterrows():
    print(f"Sample: {ix}")
    fname = "./transcriptions/" + row.path + ".json"
    with open(fname, 'r') as fh:
        transcription = json.load(fh)
    gt = row.sentence
    print(f"Groundtruth: {gt}")
    text = transcription['transcript']
    print(f"Inference: {text}")
    print("Edit Distance:", levenshteinDistance(gt, text))
    clip = listen_to_sample(df=selected, ix=ix, clips_path=clips_path)
    listen_to_split_on_boundaries(transcription['timings'], clip)
    clear_output()